# Europe’s fragmenting urban landscape using high-resolution dynamic models to characterize spatial developments 

In [ ]:
# Import required Python modules
import geopandas as gpd
import rasterio as rio
from rasterio.mask import mask
from rasterio.plot import show

import numpy as np
import seaborn as sns
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.colors import ListedColormap
from matplotlib.ticker import FormatStrFormatter
import matplotlib.patheffects as pe

from scipy.ndimage import label
from scipy.ndimage import labeled_comprehension

import pickle
import os

from tqdm import tqdm 
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

In [ ]:
# Import CCA model from previous study
import CCA_EU

# 1 Data and study area
The GHSL built-up raster data publicly available with identifiers:

- doi: 10.2905/jrc-ghsl-10007
- PID: http://data.europa.eu/89h/jrc-ghsl-10007

The functional urban area data publicly available with identifiers:

- doi: 10.2905/347F0337-F2DA-4592-87B3-E25975EC2C95
- PID: http://data.europa.eu/89h/347f0337-f2da-4592-87b3-e25975ec2c95

Country boundaries are available through GISCO: 

- https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/countries

Download the data into a (new) "..\data" folder and unzip. 

In [ ]:
# List of countries of interest
countries = np.loadtxt('parameters\europe_countries.txt', dtype=str).tolist()
countries

In [ ]:
# preprocess the input data 
ghs_built_raster_paths = [
    r'..\data\GHS_BUILT_LDS1975_GLOBE_R2018A_54009_250_V2_0\GHS_BUILT_LDS1975_GLOBE_R2018A_54009_250_V2_0.tif',
    r'..\data\GHS_BUILT_LDS1990_GLOBE_R2018A_54009_250_V2_0\GHS_BUILT_LDS1990_GLOBE_R2018A_54009_250_V2_0.tif',
    r'..\data\GHS_BUILT_LDS2000_GLOBE_R2018A_54009_250_V2_0\GHS_BUILT_LDS2000_GLOBE_R2018A_54009_250_V2_0.tif',
    r'..\data\GHS_BUILT_LDS2014_GLOBE_R2018A_54009_250_V2_0\GHS_BUILT_LDS2014_GLOBE_R2018A_54009_250_V2_0.tif']

fua_path = r'..\data\GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0\GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg'

nodata_value = -200

# Function to binary reclassify GHSL urban percentage to binary urban, using 20% threshold
def categorize_urban_vec(percent):
    threshold = 20
    out = np.full_like(percent, nodata_value)
    out[(percent >= 0) & (percent < threshold) ] = 0
    out[percent >= threshold] = 1
    return out

# Read the FUA file with global FUA's
fua_gdf = gpd.read_file(fua_path)

# Restrict dataset to countries and columns of interest
countries_gpd = fua_gdf[fua_gdf['Cntry_name'].isin(countries)]
countries_gpd = countries_gpd[['eFUA_name','Cntry_name','FUA_area','UC_area','geometry']]

#create columns - urban units at 1975, 1990, 2000, 2014,and urban growth between 75-90, 90-00, 00-14
def add_raster_to_dataframe(dataframe, raster_path, column_name):
        def process(x):
            shape = x['geometry']
            with rio.open(raster_path) as src:    
                out_img, out_transform = mask(src, [shape], crop=True)
            fua_urban_raster = np.squeeze(categorize_urban_vec(out_img))
            try:
                x[column_name] = np.unique(fua_urban_raster,return_counts=True)[1][2]
            except:
                x[column_name] = np.nan
            try:
                x['Transform'] = out_transform;
            except:
                x['Transform'] = np.nan
            try:
                x[column_name + ' raster'] = fua_urban_raster.copy();
            except:
                x[column_name + ' raster'] = np.nan
            return x
        
        dataframe = dataframe.apply(process, axis=1)
        return dataframe
        
countries_gpd = add_raster_to_dataframe(countries_gpd, ghs_built_raster_paths[0],'1975 urban')
countries_gpd = add_raster_to_dataframe(countries_gpd, ghs_built_raster_paths[1],'1990 urban')
countries_gpd = add_raster_to_dataframe(countries_gpd, ghs_built_raster_paths[2],'2000 urban')
countries_gpd = add_raster_to_dataframe(countries_gpd, ghs_built_raster_paths[3],'2014 urban')

# Remove missing data
countries_gpd.dropna(inplace=True)

# Pre-calculate urban growth quantities
countries_gpd['90-00 UG'] = countries_gpd['2000 urban']-countries_gpd['1990 urban'] 
countries_gpd['00-14 UG'] = countries_gpd['2014 urban']-countries_gpd['2000 urban'] 
countries_gpd['75-90 UG'] = countries_gpd['1990 urban']-countries_gpd['1975 urban'] 

#Keep only the FUAs with urban growth
countries_gpd = countries_gpd[(countries_gpd['75-90 UG']>0)
                              &(countries_gpd['90-00 UG']>0)
                              &(countries_gpd['00-14 UG']>0)]

In [ ]:
# Load the eurostate Countries Administrative boundaries data publicly available at
# https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/countries
administrative_units_path =r'..\data\ref-countries-2020-01m.gdb\ref-countries-2020-01m.gdb' 
eu_gdf = gpd.read_file(administrative_units_path).to_crs('ESRI:54009')

In [ ]:
# Change the GHSL names to eurostat country name
country_names = countries
country_names = [s.replace('UnitedKingdom', 'United Kingdom') for s in country_names]
country_names = [s.replace('CzechRepublic', 'Czechia') for s in country_names]
country_names = [s.replace('Turkey', 'Türkiye') for s in country_names]

# Include neighbouring countries not analyzed for mapping
non_OECDEU = ['Albania','Andorra','Armenia','Azerbaijan','Belarus','Bosnia and Herzegovina','Bulgaria',
              'Croatia','Cyprus','Georgia','Kazakhstan','Latvia','Liechtenstein','Lithuania','Malta',
              'Moldova','Monaco','Montenegro','North Macedonia','Romania','Russian Federation','San Marino',
              'Serbia','Ukraine']
bordering = ['Iran','Iraq','Syria']

EU_map_countries = country_names + non_OECDEU + bordering

# Adjust country name notation
cntyname_annotations = [((1.10768e+06, 5.61764e+06), 'Austria'),((351980, 5.93993e+06), 'Belgium'),
                        ((651122, 5.53395e+06), 'Switzerland'),((701612, 6.48093e+06), 'Denmark'),
                        ((1.69881e+06, 6.74812e+06), 'Estonia'),((1.86961e+06, 4.67712e+06), 'Greece'),
                        ((1.17062e+06, 5.84537e+06), 'Czechia'),((777853, 5.97984e+06), 'Germany'),
                        ((200000, 5.5e+06), 'France'),((1.54904e+06, 7.26525e+06), 'Finland'),
                        ((1.52867e+06, 5.57246e+06), 'Hungary'),((-593591, 6.20001e+06), 'Ireland'),
                        ((1.00735e+06, 5.08358e+06), 'Italy'),((464790, 5.85001e+06), 'Luxembourg'),
                        ((411353, 6.1078e+06), 'Netherlands'),((601144, 7.028e+06), 'Norway'),
                        ((1.43573e+06, 6.08897e+06), 'Poland'),((-728429, 4.73992e+06), 'Portugal'),
                        ((0.90898e+06, 7.09125e+06), 'Sweden'),((1.18201e+06, 5.46099e+06), 'Slovenia'),
                        ((1.50621e+06, 5.73703e+06), 'Slovakia'),((3.01291e+06, 4.68042e+06), 'Türkiye'),
                        ((-201445, 6.28578e+06), 'United Kingdom'),((-312991, 4.80445e+06), 'Spain')]

# Map the study areas
# Generate figure 3 in the article
xmin, ymin, xmax, ymax = countries_gpd.total_bounds
fig, axes = plt.subplots(1,1,figsize=(20,15))

eu_gdf[eu_gdf['NAME_ENGL'].isin(non_OECDEU + bordering)].plot(ax=axes,ec='k',linewidth=0.8,color='white')
eu_gdf[eu_gdf['NAME_ENGL'].isin(country_names)].plot(ax=axes,ec='k',linewidth=0.8,color='lightgrey')
countries_gpd.plot(ax=axes,fc='darkgrey',ec='k',linewidth=0.6)
axes.set_xlim(xmin, xmax) 
axes.set_ylim(ymin, ymax)
axes.set_facecolor('lightblue')
axes.tick_params(labelbottom=False,labelleft=False,axis=u'both', which=u'both',length=0)  
for t in cntyname_annotations :
    axes.annotate(text=t[1], xy=t[0], ha='center',fontsize=15,color='k',
                  path_effects=[pe.withStroke(linewidth=4, foreground="salmon")])

# 2 Methodology

The parameters by scenarios can be found at: https://doi.org/10.6084/m9.figshare.22194244.v1

These parameters were the results from previous study (https://doi.org/10.1016/j.compenvurbsys.2022.101892)

After downloading, copy the parameters to a "..\parameters" folder.

In [ ]:
# Reading the parameters for each scenario, this is 
# Load the parameters for modelling growth using the four spatial development scenarios 

# 0 - compact; 1 - medium compact; 2 - medium dispersed; 3 - dispersed

paras_fourscenarios = np.load(r'parameters\paras_fourscenarios.npy',allow_pickle=True)
seeds_fourscenarios = np.load(r'parameters\seeds_fourscenarios.npy',allow_pickle=True)

def get_random_parameters_for_scenario(scenario, size): 
    max_size = paras_fourscenarios[scenario].size
    nums = np.random.choice(max_size, size=size, replace=False)
    return paras_fourscenarios[scenario][nums], seeds_fourscenarios[scenario][nums] 

In [ ]:
get_random_parameters_for_scenario(1,2)

In [ ]:
# Helper functions to well-recognized spatial configuration metrics
def largest_patch_size(obs):
    obs_labeled_array, obs_num_features = label(obs)
    obs_patch_sizes = labeled_comprehension(obs,obs_labeled_array,np.arange(1, obs_num_features+1), len, float, 0)
    return np.amax(obs_patch_sizes)

def patch_size(obs):
    obs_labeled_array, obs_num_features = label(obs)
#     obs_patch_sizes = labeled_comprehension(obs,obs_labeled_array,np.arange(1, obs_num_features+1), len, float, 0)
    return obs_num_features

# Defined by https://doi.org/10.1080/17538947.2018.1474957
def DI(obs):
    obs_labeled_array, obs_num_features = label(obs)
    obs_patch_sizes = labeled_comprehension(obs,obs_labeled_array,np.arange(1, obs_num_features+1), len, float, 0)
    NP = obs_num_features
    LP = np.amax(obs_patch_sizes)/sum(obs_patch_sizes)*100
    NPN = (NP-1)/(sum(obs_patch_sizes)-1)*100
    LPN = (LP-1/sum(obs_patch_sizes))/(100-1/sum(obs_patch_sizes))*100
    return (NPN+(100-LPN))/2

#https://gist.github.com/viveksck/1110dfca01e4ec2c608515f0d5a5b1d1
def fractal_dimension(Z, threshold=0.9):

    # Only for 2d image
    assert(len(Z.shape) == 2)

    # From https://github.com/rougier/numpy-100 (#87)
    def boxcount(Z, k):
        S = np.add.reduceat(
            np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
                               np.arange(0, Z.shape[1], k), axis=1)

        # We count non-empty (0) and non-full boxes (k*k)
        return len(np.where((S > 0) & (S < k*k))[0])


    # Transform Z into a binary array
    Z = (Z < threshold)

    # Minimal dimension of image
    p = min(Z.shape)

    # Greatest power of 2 less than or equal to p
    n = 2**np.floor(np.log(p)/np.log(2))

    # Extract the exponent
    n = int(np.log(n)/np.log(2))

    # Build successive box sizes (from 2**n down to 2**1)
    sizes = 2**np.arange(n, 1, -1)

    # Actual box counting with decreasing size
    counts = []
    for size in sizes:
        counts.append(boxcount(Z, size))

    # Fit the successive log(sizes) with log (counts)
    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
    return -coeffs[0]

In [ ]:
# Create a helper function to visualize an FUA's real growth over 1975-2014 and four spatial development scenarios
# ranging from compact to dispersed
# Accompanied by the well-recognized spatial configuration metrics: fractal dimension and dispersion index

def apply_scenario_1975_2014(country, fua, scenario_no):
    country_data = countries_gpd[(countries_gpd['Cntry_name']==country)
                                &(countries_gpd['eFUA_name']==fua)]

    initial_map = country_data['1975 urban raster'].iloc[0].copy()
    rando = np.random.randint(len(paras_fourscenarios[scenario_no]))
    paras = paras_fourscenarios[scenario_no][rando]
    seed = seeds_fourscenarios[scenario_no][rando]
    total_growth = int(country_data['2014 urban'] - country_data['1975 urban'])
    rows, cols = initial_map.shape
    # set the increment size (default = 15)
    growth_per_step = 15
    # apply the model
    final_map = CCA_EU.CCA_last_snapshot([paras[0], 0, paras[1]], [0, paras[2], paras[3]],
                                         seed = seed, landmap = initial_map.copy(),
                                         rows = rows, cols = cols, urban_num = total_growth,
                                         trans_num = growth_per_step)
    return final_map

def calculate_metrics(raster):
    r = raster.copy()
    r[r==-200] = 0
    di = DI(r)
    fd = fractal_dimension(r)
    return  di, fd

def plot_20km_change_map(before, after, ax, transform, country_data):
    # Set spatial extent: 20 X 20 km
    
    x = float(country_data['geometry'].centroid.x)
    y = float(country_data['geometry'].centroid.y)
    xmin, ymin, xmax, ymax = x-10000,y-10000,x+10000,y+10000
    
    changes = 2 * before + after
    changes[ (after == -200) | (before == -200)] = 0
    
    cmap = ListedColormap(["white","k","grey", "grey"])
    # Legend:
    # 0 : non-built to non-built (white)
    # 1 : non-built to built (black)
    # 2 : built to non-built (should not exist, grey)
    # 3 : built to built (grey)
    
    ax.set_xlim(xmin,xmax)
    ax.set_ylim(ymin,ymax)
    ax.tick_params(labelbottom=False,labelleft=False,axis=u'both', which=u'both',length=0)
       
    country_data.boundary.plot(ax=ax,ec='darkgrey',linewidth=2)
    
    #interpolation = "nearest" turns of anti-aliasing of the pixels
    show(changes, ax=ax, cmap=cmap,transform=transform,interpolation="nearest")
    
def plot_fua_scenarios(country,fua):
    country_data = countries_gpd[(countries_gpd['Cntry_name']==country)
                                &(countries_gpd['eFUA_name']==fua)]
    initial_map = country_data['1975 urban raster'].iloc[0]
    
    # calculate metrics for initial map
    di_pre, fd_pre = calculate_metrics(initial_map);

    
    fig, axes = plt.subplots(5,1,figsize=(3,13))
    
    stat_superscripts = ['Obs','Cmp', 'MedCmpt', 'MedDisp', 'Disp']

    for i in range(5):
        if i == 0:
            result_map = country_data['2014 urban raster'].iloc[0]
        else:
            scenario_no = i - 1
            result_map = apply_scenario_1975_2014(country, fua, scenario_no)
         
        di, fd = calculate_metrics(result_map)
        plot_20km_change_map(initial_map, result_map, axes[i], country_data['Transform'].iloc[0], country_data)
        
        if i==0:
            axes[i].set_title('$DI^{Obs}_{1975}=%.2f \quad FD^{Obs}_{1975}=%.2f$ \n'%(di_pre,fd_pre) + 
                              '$DI^{Obs}_{2014}=%.2f \quad FD^{Obs}_{2014}=%.2f$'%(di,fd))
        else: 
            axes[i].set_title('$DI^{'+ stat_superscripts[i] +'}_{2014}=%.2f$'%(di) +
                              '$\quad FD^{'+ stat_superscripts[i] +'}_{2014}=%.2f$'%(fd))
        
    fig.tight_layout()

In [ ]:
# Generate figure 4 & 5 in the article
plot_fua_scenarios('Portugal','Braga')

# 3. Result analysis

In [ ]:
#This is from MCMCABC and CCA and should be probably imported rather than copied

from scipy.signal import fftconvolve
from scipy.stats import ks_2samp

def kernel_expo_square(cutoff,beta):
    '''
    Input: 1. exponential decay cutoff value - cutoff, 2. exponential lambda - beta;
    Output: a square exponential decay kernel matrix, 
            each entry has the exponential distance decay value from the square kernel's centre, normalised by the sum of the kernel.
    Algorithm:
    1. Given cutoff, calculate the distance where the cutoff value is reached, ceil round to the nearest larger integer a;
    2. Using numpy.meshgrid to generate distance matrix cooridnates; vectorization calculate distance of each cell to the centre of the square;
    3. Apply the exponential decay function to the distances matrix and return the normalized exponential decay kernel matrix.
    '''
    a = int(np.ceil(np.log(cutoff)/(-beta)))
    centre_row, centre_col = a, a
    xv, yv = np.meshgrid(np.arange(2*a+1),np.arange(2*a+1))
    d_celltocentre = ((xv-centre_row)**2+(yv-centre_col)**2)**0.5
    kernel = np.exp(-beta*d_celltocentre)
    return kernel/sum(sum(kernel))

kernels=[kernel_expo_square(0.01,beta) for beta in [0.2,0.5,2.0]]

#function to measure the difference between an observation and a simulation map
def ks_dis(obs,sim):
    """
    This function measures the difference between an observation and a simulation map
    using KS statistic.
    Input: an observation and a simulation map - numpy arrays
    Output: the sum of KS statistics on three spatial scales.
    """
    obs_urbandensity0 = fftconvolve(obs,kernels[0],mode='same')[obs==1].flatten()
    obs_urbandensity1 = fftconvolve(obs,kernels[1],mode='same')[obs==1].flatten()
    obs_urbandensity2 = fftconvolve(obs,kernels[2],mode='same')[obs==1].flatten()
    simulation_urbandensity0 = fftconvolve(sim,kernels[0],mode='same')[sim==1].flatten()
    statistic0, pvalue = ks_2samp(obs_urbandensity0,simulation_urbandensity0)
    simulation_urbandensity2 = fftconvolve(sim,kernels[2],mode='same')[sim==1].flatten()
    statistic2, pvalue = ks_2samp(obs_urbandensity2,simulation_urbandensity2)
    simulation_urbandensity1 = fftconvolve(sim,kernels[1],mode='same')[sim==1].flatten()
    statistic1, pvalue = ks_2samp(obs_urbandensity1,simulation_urbandensity1)
    return (statistic0 + statistic2 + statistic1) 

def calculate_distance(observed, simulated):
    o = observed.copy()
    s = simulated.copy()
    o[o == -200] = 0
    s[s == -200] = 0
    return ks_dis(o,s)

In [ ]:
# For each FUA and Period:
#   Do 4 X 60 runs 
#   Calcute GOF
#   Select 60 best runs
#   Calculate percentage for each (soft classify)
#   Identify best fit (hard classify)

def run_model(before, after, param_set, seed):
    n_before = np.count_nonzero(before == 1);
    n_after = np.count_nonzero(after == 1);
    total_growth = int(n_after - n_before)
    rows, cols = before.shape
    # set the increment size (default = 15)
    growth_per_step = 15
    # apply the model
    final_map = CCA_EU.CCA_last_snapshot([param_set[0], 0, param_set[1]], [0, param_set[2], param_set[3]],
                                         seed = seed, landmap = before.copy(),
                                         rows = rows, cols = cols, urban_num = total_growth,
                                         trans_num = growth_per_step)
    
    gof = calculate_distance(final_map, after) 
    return gof

def run_period_single_fua(before, after):
        gof = []
        for s in range(4):
            param_sets, seeds = get_random_parameters_for_scenario(s,60)
            for p in range(60):
                gof.append(run_model(before, after, param_sets[p], seeds[p]))

        counts, bins = np.histogram(np.floor(np.argsort(gof)[:60]/60), range(5))  
        soft_class = counts / 60
        soft_class_dispersed = soft_class[3]
        hard_class = np.argmax(counts)
        return hard_class, soft_class, soft_class_dispersed

def run_genesis_single_fua(after):
    before = after.copy();
    before[before==1] = 0;
    return run_period_single_fua(before, after)

def run_period(fua_row, before_str, after_str):
    before = fua_row[before_str + ' urban raster'];
    after = fua_row[after_str + ' urban raster'];
    hard_class, soft_class, soft_class_dispersed = run_period_single_fua(before, after);
    
    return {before_str + '-' + after_str + ' dominant mode': hard_class,
            before_str + '-' + after_str + ' dispersed': soft_class_dispersed}


def run_genesis(fua_row, after_str):
    after = fua_row[after_str + ' urban raster'];
    hard_class, soft_class, soft_class_dispersed = run_genesis_single_fua(after);
    return {'0-' + after_str + ' dominant mode': hard_class,
            '0-' + after_str + ' dispersed': soft_class_dispersed}

def run_period_multiple_fua(dataframe, before_str, after_str):
    applied_df = dataframe.progress_apply(lambda row: run_period(row, before_str, after_str), axis='columns', result_type='expand')
    dataframe = pd.concat([dataframe, applied_df], axis='columns')
    return dataframe 

def run_genesis_multiple_fua(dataframe, after_str):
    applied_df = dataframe.progress_apply(lambda row: run_genesis(row, after_str), axis='columns', result_type='expand')
    dataframe = pd.concat([dataframe, applied_df], axis='columns')
    return dataframe 

In [ ]:
# This cell is doing extensive analysis: 240 model runs for all FUA and for four periods. This may take days.

print("0-1975")
countries_gpd = run_genesis_multiple_fua(countries_gpd,'1975')
print("1975-1990")
countries_gpd = run_period_multiple_fua(countries_gpd,'1975','1990')
print("1990-2000")
countries_gpd = run_period_multiple_fua(countries_gpd,'1990','2000')
print("2000-2014")
countries_gpd = run_period_multiple_fua(countries_gpd,'2000','2014')

# After all this work, make sure to store it for retrieval later
countries_gpd.to_pickle("europe_results_countries_gpd.pkl")

In [ ]:
countries_gpd = pd.read_pickle("europe_results_countries_gpd.pkl")

In [ ]:
# Process for plotting
values_075_avg = np.histogram(countries_gpd["0-1975 dominant mode"].to_numpy(),range(5))
values_7590_avg = np.histogram(countries_gpd["1975-1990 dominant mode"].to_numpy(),range(5))
values_9000_avg = np.histogram(countries_gpd["1990-2000 dominant mode"].to_numpy(),range(5))
values_0014_avg = np.histogram(countries_gpd["2000-2014 dominant mode"].to_numpy(),range(5))

countries_gpd['075color'] = countries_gpd["0-1975 dominant mode"].apply(
        lambda x: (x == 3 and 'blue') or (x == 2 and 'orange') \
        or (x == 1 and 'brown') or (x == 0 and 'green')) 
countries_gpd['7590color'] = countries_gpd["1975-1990 dominant mode"].apply(
        lambda x: (x == 3 and 'blue') or (x == 2 and 'orange') \
        or (x == 1 and 'brown') or (x == 0 and 'green')) 
countries_gpd['9000color'] = countries_gpd["1990-2000 dominant mode"].apply(
        lambda x: (x == 3 and 'blue') or (x == 2 and 'orange') \
        or (x == 1 and 'brown') or (x == 0 and 'green')) 
countries_gpd['0014color'] = countries_gpd["2000-2014 dominant mode"].apply(
        lambda x: (x == 3 and 'blue') or (x == 2 and 'orange') \
        or (x == 1 and 'brown') or (x == 0 and 'green')) 

In [ ]:
countries_gpd['more_dispersed0014than7500'] = countries_gpd.apply(
        lambda x: (x['1990-2000 dispersed'] > x['2000-2014 dispersed']  and 'blue') or 
    (x['1990-2000 dispersed'] <= x['2000-2014 dispersed'] and 'yellow'), axis='columns')

In [ ]:
def plot_classification_map(colour_column,piechart_data, title): 
    xmin, ymin, xmax, ymax = countries_gpd.total_bounds
    fig, axes = plt.subplots(1,1,figsize=(20,15))

    eu_gdf[eu_gdf['NAME_ENGL'].isin(non_OECDEU + bordering)].plot(ax=axes,ec='k',linewidth=0.8,color='white')
    eu_gdf[eu_gdf['NAME_ENGL'].isin(country_names)].plot(ax=axes,ec='k',linewidth=0.8,color='lightgrey')
    for t in cntyname_annotations :
        axes.annotate(text=t[1], xy=t[0], ha='center',fontsize=15)

    countries_gpd.plot(ax=axes,color=countries_gpd[colour_column],ec='grey',linewidth=0.3)

    left, bottom, width, height = [0.1, 0.7, 0.14, 0.14]
    ax2 = fig.add_axes([left, bottom, width, height])
    ax2.pie(piechart_data, colors=('green','brown', 'orange', 'blue'),autopct='%.0f%%',textprops={'fontsize': 20},pctdistance=1.3)
    title='1975-1990'
    plt.text(0.2, 0.95, title,horizontalalignment='center',fontsize=20,transform = axes.transAxes)
    axes.set_xlim(xmin, xmax) 
    axes.set_ylim(ymin, ymax)
    axes.axis('off')

In [ ]:
plot_classification_map('075color', values_075_avg[0], '"0"-1975')

In [ ]:
plot_classification_map('7590color', values_7590_avg[0], '1975-1990')

In [ ]:
plot_classification_map('9000color', values_9000_avg[0], '1990-2000')

In [ ]:
plot_classification_map('0014color', values_0014_avg[0], '2000-2014')

In [ ]:
# Generate figure 2
counts = countries_gpd['more_dispersed0014than7500'].value_counts()
num_blue_avg = 0
num_yellow_avg = 0
if 'yellow' in counts:
    num_yellow_avg = counts['yellow']

if 'blue' in counts:
    num_blue_avg = counts['blue']
    
pct_blue_avg = num_blue_avg / (num_green_avg + num_blue_avg)


xmin, ymin, xmax, ymax = countries_gpd.total_bounds
fig, axes = plt.subplots(1,1,figsize=(20,13))
eu_gdf[eu_gdf['NAME_ENGL'].isin(non_OECDEU+bordering)].plot(ax=axes,ec='k',linewidth=0.8,color='white')
eu_gdf[eu_gdf['NAME_ENGL'].isin(country_names)].plot(ax=axes,ec='k',linewidth=0.8,color='lightgrey')
for t in cntyname_annotations :
    axes.annotate(text=t[1], xy=t[0], ha='center',fontsize=15)

countries_gpd.plot(ax=axes,color=countries_gpd['more_dispersed0014than7500'],ec='grey',linewidth=0.3)

green_patch = mpatches.Patch(color='green', label='2000-2014 dispersion < 1975-1990:\n %d' % (num_green_avg))
blue_patch = mpatches.Patch(color='blue', label='2000-2014 dispersion > 1975-1990:\n %d' % (num_blue_avg))
_ = fig.legend(handles=[green_patch,blue_patch], bbox_to_anchor=(0.48,0.88),fontsize=18, ncol=1,
               title=r" %.0f%% more dispersed in 2000-2014 than 1975-1990"  % (pct_blue_avg),
               title_fontsize=18)

axes.set_xlim(xmin, xmax) 
axes.set_ylim(ymin, ymax)
axes.axis('off')